In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from typing import Dict, Any, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# ========== PREPARAÇÃO DE DADOS ==========

def get_clean_data(df: pd.DataFrame, target_col: str = "Vazao_BBR") -> pd.DataFrame:
    """
    Remove apenas linhas com valores inválidos (-1) no target.
    """
    df_clean = df[df[target_col] != -1].copy().reset_index(drop=True)
    print(f"  [DADOS LIMPOS] {len(df_clean)} amostras válidas (removidos {len(df) - len(df_clean)} com -1)")
    return df_clean

def apply_random_mask(df: pd.DataFrame, missing_fraction: float, seed: int = None) -> pd.DataFrame:
    """
    Aplica máscara aleatória para BASELINES - marca valores a serem 'escondidos' para teste.
    """
    df_masked = df.copy()
    n_samples = len(df_masked)
    n_mask = max(1, int(missing_fraction * n_samples))
    
    if seed is not None:
        np.random.seed(seed)
    
    mask_indices = np.random.choice(df_masked.index, size=n_mask, replace=False)
    df_masked['mask_applied'] = 0
    df_masked.loc[mask_indices, 'mask_applied'] = 1
    
    print(f"    Máscara aplicada: {n_mask}/{n_samples} amostras ({missing_fraction*100:.0f}%)")
    return df_masked

# ========== FEATURE ENGINEERING ==========

def engineer_features_for_imputation(df: pd.DataFrame, target_col: str = "Vazao_BBR") -> pd.DataFrame:
    """
    Feature engineering SEM data leakage para imputação.
    Regras:
    - Nunca usar o valor ATUAL do target para criar features.
    - Substitui inf/NaN por valores numéricos seguros.
    """
    df = df.copy()
    
    # ✅ Garantir que Data está como datetime e criar features temporais básicas
    if 'Data' in df.columns:
        df['Data'] = pd.to_datetime(df['Data'])
        df['hour'] = df['Data'].dt.hour
        df['day_of_week'] = df['Data'].dt.dayofweek
        df['day_of_month'] = df['Data'].dt.day

    # ✅ Features derivadas de outras colunas (não do target)
    df["Atraso_log"] = np.log1p(df["Atraso(ms)"].clip(lower=0))
    df["Hop_inv"] = 1 / (df["Hop_count"] + 1)
    df["Atraso_x_Hop"] = df["Atraso(ms)"] * df["Hop_count"]
    df["Atraso_sq"] = df["Atraso(ms)"] ** 2
    df["Hop_sq"] = df["Hop_count"] ** 2

    if 'hour' in df.columns:
        df["Atraso_x_hour"] = df["Atraso(ms)"] * df["hour"]
        df["Hop_x_hour"] = df["Hop_count"] * df["hour"]
        df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
        df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)

    # ✅ Features do target baseadas em valores passados
    valid_mask = df[target_col] != -1
    target_series = df[target_col].copy()
    target_series[~valid_mask] = np.nan

    # Lags
    for lag in [1, 2, 3, 6]:
        df[f"Vazao_lag{lag}"] = target_series.shift(lag)

    # Diferenças e variações percentuais
    df["Vazao_diff1"] = target_series.diff(1)
    df["Vazao_diff2"] = target_series.diff(2)
    df["Vazao_pct_change"] = target_series.pct_change()

    # Rolling features (sem vazamento)
    for w in [3, 6]:
        shifted = target_series.shift(1)
        df[f"Vazao_roll_mean_{w}"] = shifted.rolling(window=w, min_periods=w).mean()
        df[f"Vazao_roll_std_{w}"] = shifted.rolling(window=w, min_periods=w).std()
    df["Vazao_roll_max_6"] = shifted.rolling(window=6, min_periods=6).max()
    df["Vazao_roll_min_6"] = shifted.rolling(window=6, min_periods=6).min()

    # Relações derivadas
    lag1 = target_series.shift(1)
    df["Vazao_lag1_div_Atraso"] = lag1 / (df["Atraso(ms)"] + 1)
    df["Vazao_lag1_div_Hops"] = lag1 / (df["Hop_count"] + 1)
    df["Efficiency_lag1"] = lag1 / ((df["Atraso(ms)"] + 1) * (df["Hop_count"] + 1))

    # Transformações seguras
    df["Vazao_lag1_log"] = np.log1p(lag1.clip(lower=0))
    df["Vazao_lag1_sqrt"] = np.sqrt(lag1.clip(lower=0))

    # Estatísticas de janela expandida
    df["Vazao_expanding_mean"] = target_series.shift(1).expanding(min_periods=1).mean()
    df["Vazao_expanding_std"] = target_series.shift(1).expanding(min_periods=3).std()

    # Medidas globais do target
    df['Feature_Vazao_bbr_median'] = target_series.median()
    df['Feature_Vazao_bbr_mean'] = target_series.mean()

    # Restaurar valores -1 originais
    df.loc[~valid_mask, target_col] = -1

    # ✅ Tratamento final: substituir infinitos e NaNs
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    for col in df.select_dtypes(include=[np.number]).columns:
        if col == target_col:
            continue
        if df[col].isna().any():
            if any(k in col for k in ['lag', 'roll', 'diff', 'pct', 'expanding']):
                df[col].fillna(0, inplace=True)
            else:
                df[col].fillna(df[col].median(), inplace=True)

    return df


# ========== CÁLCULO DE MÉTRICAS ==========

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray, prediction_time: float = None) -> Dict[str, Any]:
    """
    Calcula métricas de regressão de forma robusta.
    """
    # Garantir arrays numpy
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    
    # Remover valores inválidos
    mask = ~(np.isnan(y_true) | np.isnan(y_pred) | np.isinf(y_true) | np.isinf(y_pred))
    
    if mask.sum() < 2:
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}
    
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    try:
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        nrmse = (rmse / (np.mean(y_true) + 1e-8)) * 100
        rmse_normalized = rmse / 1_000_000  # Para milhões
        
        r2 = r2_score(y_true, y_pred)
        r2 = r2 if not np.isnan(r2) and np.isfinite(r2) else None
        
        mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
        
        # Calcular tempo por amostra (em milissegundos)
        time_per_sample = None
        if prediction_time is not None and len(y_true) > 0:
            time_per_sample = round((prediction_time / len(y_true)) * 1000, 4)  # ms por amostra
        
        return {
            "rmse": round(rmse_normalized, 2),
            "nrmse": round(nrmse, 2),
            "r2": r2,
            "mape": round(mape, 2),
            "prediction_time_per_sample": time_per_sample  # ms por amostra
        }
    except:
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}

# ========== AVALIAÇÃO BASELINES ==========

def evaluate_baselines(df_clean: pd.DataFrame, missing_fraction: float, target_col: str = "Vazao_BBR") -> Dict:
    """
    Avalia métodos baseline:
    1. Aplica máscara aleatória (esconde valores)
    2. Salva valores originais
    3. Imputa com cada método
    4. Calcula métricas comparando com valores originais
    """
    print(f"  [BASELINE] Avaliando fração {missing_fraction:.0%}")
    
    # Aplicar máscara
    df_masked = apply_random_mask(df_clean, missing_fraction, seed=42)
    
    # Guardar valores originais
    mask_indices = df_masked[df_masked['mask_applied'] == 1].index
    y_true = df_masked.loc[mask_indices, target_col].values
    
    results = {}
    
    # Criar cópia com NaN nos valores mascarados
    df_with_nan = df_masked.copy()
    df_with_nan.loc[mask_indices, target_col] = np.nan
    
    # ===== MÉDIA =====
    try:
        df_mean = df_with_nan.copy()
        mean_value = df_mean[target_col].mean()
        df_mean[target_col] = df_mean[target_col].fillna(mean_value)
        y_pred = df_mean.loc[mask_indices, target_col].values
        results['Mean'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['Mean'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== MEDIANA =====
    try:
        df_median = df_with_nan.copy()
        median_value = df_median[target_col].median()
        df_median[target_col] = df_median[target_col].fillna(median_value)
        y_pred = df_median.loc[mask_indices, target_col].values
        results['Median'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['Median'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== KNN IMPUTER =====
    try:
        df_knn = df_with_nan.copy()
        imputer = KNNImputer(n_neighbors=min(5, len(df_clean) // 2))
        df_knn[target_col] = imputer.fit_transform(df_knn[[target_col]]).ravel()
        y_pred = df_knn.loc[mask_indices, target_col].values
        results['KNNImputer'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['KNNImputer'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== FORWARD FILL =====
    try:
        df_ffill = df_with_nan.copy()
        df_ffill[target_col] = df_ffill[target_col].ffill().bfill()
        y_pred = df_ffill.loc[mask_indices, target_col].values
        results['ForwardFill'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['ForwardFill'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== BACKWARD FILL =====
    try:
        df_bfill = df_with_nan.copy()
        df_bfill[target_col] = df_bfill[target_col].bfill().ffill()
        y_pred = df_bfill.loc[mask_indices, target_col].values
        results['BackwardFill'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['BackwardFill'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== ROLLING MEAN =====
    try:
        df_rolling = df_with_nan.copy()
        df_rolling[target_col] = df_rolling[target_col].rolling(window=3, min_periods=1).mean()
        df_rolling[target_col] = df_rolling[target_col].ffill().bfill()
        y_pred = df_rolling.loc[mask_indices, target_col].values
        results['RollingMean'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['RollingMean'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    # ===== INTERPOLAÇÃO LINEAR =====
    try:
        df_linear = df_with_nan.copy()
        df_linear[target_col] = df_linear[target_col].interpolate(method='linear').ffill().bfill()
        y_pred = df_linear.loc[mask_indices, target_col].values
        results['LinearInterpolation'] = calculate_metrics(y_true, y_pred)
    except Exception as e:
        results['LinearInterpolation'] = {"rmse": None, "nrmse": None, "r2": None, "mape": None}
    
    return results

# ========== AVALIAÇÃO STACKING COM MISSING DATA SIMULADO ==========

def evaluate_stacking_with_missing(df_clean: pd.DataFrame, missing_fraction: float, target_col: str = "Vazao_BBR") -> Dict:
    """
    AVALIA STACKING COM MISSING DATA SIMULADO - igual às baselines
    """
    print(f"  [STACKING] Avaliando com missing_fraction={missing_fraction:.0%}")
    
    # 1. APLICAR MÁSCARA ALEATÓRIA (igual às baselines)
    df_masked = apply_random_mask(df_clean, missing_fraction, seed=42)
    mask_indices = df_masked[df_masked['mask_applied'] == 1].index
    
    # 2. SEPARAR DADOS
    # Treino: dados sem máscara (valores conhecidos)
    df_train = df_masked[df_masked['mask_applied'] == 0].copy()
    # Teste: dados com máscara (valores "faltantes" que queremos prever)
    df_test = df_masked[df_masked['mask_applied'] == 1].copy()
    
    print(f"    Treino: {len(df_train)} amostras, Teste: {len(df_test)} amostras")
    
    if len(df_train) < 10 or len(df_test) < 5:
        print(f"    ⚠️ Dados insuficientes")
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}
    
    # 3. FEATURE ENGINEERING SEPARADA
    df_train_feat = engineer_features_for_imputation(df_train, target_col)
    df_test_feat = engineer_features_for_imputation(df_test, target_col)
    
    # 4. PREPARAR FEATURES
    exclude_cols = {target_col, 'Data', 'mask_applied'}
    feature_cols = [c for c in df_train_feat.columns if c not in exclude_cols]
    
    # Limpar NaN
    df_train_feat = df_train_feat.dropna(subset=feature_cols + [target_col])
    df_test_feat = df_test_feat.dropna(subset=feature_cols)
    
    if df_train_feat.empty or df_test_feat.empty:
        print(f"    ⚠️ Dados insuficientes após limpeza")
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}
    
    # 5. TREINAR E PREDIZER
    X_train = df_train_feat[feature_cols].fillna(0).values
    y_train = df_train_feat[target_col].values
    X_test = df_test_feat[feature_cols].fillna(0).values
    
    # Valores verdadeiros para cálculo de métricas
    y_true = df_clean.loc[mask_indices, target_col].values
    
    print(f"    Modelo: {len(X_train)} -> {len(X_test)} previsões")
    
    try:
        import time
        # Escalonamento
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_test_scaled = scaler_X.transform(X_test)
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        
        # Modelos base
        base_models = [
            ('xgb', XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1,
                                random_state=42, verbosity=0)),
            ('rf', RandomForestRegressor(n_estimators=100, max_depth=10,
                                        random_state=42, n_jobs=-1)),
            ('gb', GradientBoostingRegressor(n_estimators=100, max_depth=3,
                                            learning_rate=0.1, random_state=42)),
            ('knn', KNeighborsRegressor(n_neighbors=min(5, len(X_train)//3), 
                                       weights='distance')),
        ]
        
        # Stacking
        stacking = StackingRegressor(
            estimators=base_models,
            final_estimator=Ridge(alpha=1.0),
            cv=min(3, len(X_train)//10),
            n_jobs=-1
        )
        
        # Treinar
        stacking.fit(X_train_scaled, y_train_scaled)
        
        # Medir tempo de predição
        start_time = time.time()
        y_pred_scaled = stacking.predict(X_test_scaled)
        prediction_time = time.time() - start_time
        
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
        
        # Calcular métricas
        metrics = calculate_metrics(y_true, y_pred, prediction_time)
        
        print(f"    ✅ RMSE: {metrics['rmse']:.2f}M, R²: {metrics['r2']:.4f}")
        
        return metrics
        
    except Exception as e:
        print(f"    ❌ Erro no stacking: {e}")
        return {"rmse": None, "nrmse": None, "r2": None, "mape": None, "prediction_time_per_sample": None}

# ========== PIPELINE COMPLETO DE AVALIAÇÃO ==========

def evaluate_file(file_path: Path, missing_fractions: List[float]) -> Dict:
    """
    Pipeline completo de avaliação para um arquivo CSV.
    """
    print(f"\n{'='*80}")
    print(f"[AVALIANDO] {file_path.name}")
    print(f"{'='*80}")
    
    # Carregar dados
    df = pd.read_csv(file_path)
    source = file_path.stem.replace("_merged", "").replace("_largest_subseries", "")
    target_col = "Vazao_BBR"
    
    # Limpar dados (remover apenas -1)
    df_clean = get_clean_data(df, target_col)
    
    if len(df_clean) < 20:
        print(f"  ⚠️ Dados insuficientes: apenas {len(df_clean)} amostras")
        return None
    
    results = {}
    
    # Avaliar para cada fração de missing
    for frac in missing_fractions:
        print(f"\n  {'─'*60}")
        print(f"  FRAÇÃO DE MISSING: {frac:.0%}")
        print(f"  {'─'*60}")
        
        # 1. BASELINES: máscara aleatória + imputação
        baseline_results = evaluate_baselines(df_clean, frac, target_col)
        
        # 2. STACKING: avaliação com missing data simulado (igual às baselines)
        stacking_results = evaluate_stacking_with_missing(df_clean, frac, target_col)
        
        # Armazenar resultados
        results[str(frac)] = {
            "baseline": baseline_results,
            "stacking": {
                "mean": {
                    "StackingRegressor": stacking_results
                }
            }
        }
    
    return {
        "source": source,
        "results": results,
        "n_samples": len(df_clean)
    }

# ========== ANÁLISE DE DESEMPENHO ==========

def analyze_stacking_performance(results: Dict) -> Dict[str, Any]:
    """
    Analisa se o stacking foi melhor que as baselines.
    Retorna estatísticas de vitórias e métricas médias.
    """
    stacking_wins = 0
    total_comparisons = 0
    stacking_rmse_list = []
    best_baseline_rmse_list = []
    prediction_times = []  # ✅ Nova lista
    
    for frac, data in results.items():
        baseline_data = data.get("baseline", {})
        stacking_data = data.get("stacking", {}).get("mean", {}).get("StackingRegressor", {})
        
        if not baseline_data or not stacking_data:
            continue
        
        stacking_rmse = stacking_data.get("rmse")
        if stacking_rmse is None:
            continue
        
        # ✅ Coletar tempo de predição
        pred_time = stacking_data.get("prediction_time_per_sample")
        if pred_time is not None:
            prediction_times.append(pred_time)
        
        # Encontrar melhor baseline
        baseline_rmses = [
            metrics["rmse"] for metrics in baseline_data.values() 
            if metrics.get("rmse") is not None
        ]
        
        if not baseline_rmses:
            continue
        
        best_baseline_rmse = min(baseline_rmses)
        
        total_comparisons += 1
        stacking_rmse_list.append(stacking_rmse)
        best_baseline_rmse_list.append(best_baseline_rmse)
        
        if stacking_rmse < best_baseline_rmse:
            stacking_wins += 1
    
    if total_comparisons == 0:
        return {
            "should_impute": False,
            "win_rate": 0.0,
            "avg_improvement": 0.0,
            "total_comparisons": 0,
            "avg_prediction_time_per_sample": None
        }
    
    win_rate = stacking_wins / total_comparisons
    avg_stacking_rmse = np.mean(stacking_rmse_list)
    avg_best_baseline_rmse = np.mean(best_baseline_rmse_list)
    avg_improvement = ((avg_best_baseline_rmse - avg_stacking_rmse) / avg_best_baseline_rmse) * 100
    
    # ✅ Calcular tempo médio
    avg_pred_time = round(np.mean(prediction_times), 4) if prediction_times else None
    
    return {
        "should_impute": win_rate >= 0.5,  # Stacking melhor em >= 50% dos casos
        "win_rate": win_rate,
        "avg_improvement": avg_improvement,
        "total_comparisons": total_comparisons,
        "stacking_wins": stacking_wins,
        "avg_stacking_rmse": avg_stacking_rmse,
        "avg_baseline_rmse": avg_best_baseline_rmse,
        "avg_prediction_time_per_sample": avg_pred_time  # ✅ ms por amostra
    }

# ========== IMPUTAÇÃO DE DADOS ==========

def impute_with_baselines(df: pd.DataFrame, target_col: str, output_dir: Path, source: str):
    """
    Imputa valores -1 usando métodos baseline.
    """
    mask_missing = df[target_col] == -1
    n_missing = mask_missing.sum()
    
    if n_missing == 0:
        return
    
    print(f"    [BASELINES] Imputando {n_missing} valores...")
    
    # Dados válidos para calcular estatísticas
    df_clean = df[df[target_col] != -1].copy()
    
    # Adicionar coluna de rastreamento
    cols_to_save = ["Data", "Atraso(ms)", "Hop_count", "Bottleneck", target_col, "is_imputed"]
    
    # MÉDIA
    try:
        df_mean = df.copy()
        df_mean['is_imputed'] = 0
        df_mean.loc[mask_missing, 'is_imputed'] = 1
        df_mean.loc[mask_missing, target_col] = df_clean[target_col].mean()
        df_mean[cols_to_save].to_csv(output_dir / f"{source}_baseline_mean.csv", index=False)
        print(f"      ✓ Mean: {output_dir / f'{source}_baseline_mean.csv'}")
    except Exception as e:
        print(f"      ✗ Mean falhou: {e}")
    
    # MEDIANA
    try:
        df_median = df.copy()
        df_median['is_imputed'] = 0
        df_median.loc[mask_missing, 'is_imputed'] = 1
        df_median.loc[mask_missing, target_col] = df_clean[target_col].median()
        df_median[cols_to_save].to_csv(output_dir / f"{source}_baseline_median.csv", index=False)
        print(f"      ✓ Median: {output_dir / f'{source}_baseline_median.csv'}")
    except Exception as e:
        print(f"      ✗ Median falhou: {e}")
    
    # KNN
    try:
        df_knn = df.copy()
        df_knn['is_imputed'] = 0
        df_knn.loc[mask_missing, 'is_imputed'] = 1
        imputer = KNNImputer(n_neighbors=min(5, len(df_clean) // 2))
        df_knn[target_col] = imputer.fit_transform(df[[target_col]].replace(-1, np.nan)).ravel()
        df_knn[cols_to_save].to_csv(output_dir / f"{source}_baseline_knn.csv", index=False)
        print(f"      ✓ KNN: {output_dir / f'{source}_baseline_knn.csv'}")
    except Exception as e:
        print(f"      ✗ KNN falhou: {e}")
    
    # FORWARD FILL
    try:
        df_ffill = df.copy()
        df_ffill['is_imputed'] = 0
        df_ffill.loc[mask_missing, 'is_imputed'] = 1
        df_ffill[target_col] = df_ffill[target_col].replace(-1, np.nan).ffill().bfill()
        df_ffill[cols_to_save].to_csv(output_dir / f"{source}_baseline_ffill.csv", index=False)
        print(f"      ✓ ForwardFill: {output_dir / f'{source}_baseline_ffill.csv'}")
    except Exception as e:
        print(f"      ✗ ForwardFill falhou: {e}")
    
    # BACKWARD FILL
    try:
        df_bfill = df.copy()
        df_bfill['is_imputed'] = 0
        df_bfill.loc[mask_missing, 'is_imputed'] = 1
        df_bfill[target_col] = df_bfill[target_col].replace(-1, np.nan).bfill().ffill()
        df_bfill[cols_to_save].to_csv(output_dir / f"{source}_baseline_bfill.csv", index=False)
        print(f"      ✓ BackwardFill: {output_dir / f'{source}_baseline_bfill.csv'}")
    except Exception as e:
        print(f"      ✗ BackwardFill falhou: {e}")

def impute_with_stacking(df: pd.DataFrame, target_col: str, output_dir: Path, source: str):
    """
    Imputa valores -1 usando Stacking Regressor - VERSÃO MELHORADA.
    """
    mask_missing = df[target_col] == -1
    n_missing = mask_missing.sum()
    
    if n_missing == 0:
        return
    
    print(f"    [STACKING] Imputando {n_missing} valores...")
    
    try:
        # Separar dados válidos e missing
        df_clean = df[df[target_col] != -1].copy()
        
        if len(df_clean) < 10:
            print(f"      ✗ Dados insuficientes para treinar stacking")
            return
        
        # Feature engineering MELHORADO nos dados válidos
        df_clean_feat = engineer_features_for_imputation(df_clean, target_col)
        
        # Preparar features (excluir colunas não numéricas e target)
        exclude_cols = {target_col, 'Data', 'mask_applied'}
        feature_cols = [c for c in df_clean_feat.columns 
                       if c not in exclude_cols and pd.api.types.is_numeric_dtype(df_clean_feat[c])]
        
        X_train = df_clean_feat[feature_cols].fillna(0).values
        y_train = df_clean_feat[target_col].values
        
        print(f"      Treinando com {len(X_train)} amostras e {len(feature_cols)} features")
        
        # Treinar modelo completo
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()
        
        # Modelos base com mais diversidade
        base_models = [
            ('xgb', XGBRegressor(n_estimators=150, max_depth=4, learning_rate=0.05, 
                                random_state=42, verbosity=0, subsample=0.8)),
            ('rf', RandomForestRegressor(n_estimators=150, max_depth=15, 
                                         random_state=42, n_jobs=-1, min_samples_split=3)),
            ('gb', GradientBoostingRegressor(n_estimators=150, max_depth=4, 
                                            learning_rate=0.05, random_state=42, subsample=0.8)),
            ('knn', KNeighborsRegressor(n_neighbors=min(7, len(X_train) // 4), 
                                       weights='distance', p=1)),
        ]
        
        # Meta-modelo com regularização
        stacking = StackingRegressor(
            estimators=base_models,
            final_estimator=Ridge(alpha=10.0),  # Mais regularização
            cv=min(5, len(X_train) // 10),
            n_jobs=-1,
            passthrough=True  # Incluir features originais
        )
        
        print(f"      Treinando ensemble...")
        stacking.fit(X_train_scaled, y_train_scaled)
        
        # IMPUTAR LINHA POR LINHA para capturar dependências temporais
        df_imputed = df.copy()
        df_imputed['is_imputed'] = 0
        
        missing_indices = df[mask_missing].index.tolist()
        imputed_values = []
        
        print(f"      Imputando {len(missing_indices)} valores...")
        
        for idx in missing_indices:
            # Criar DataFrame temporário até este ponto
            df_temp = df_imputed.iloc[:idx+1].copy()
            
            # Para a última linha, usar valor já imputado (se houver) ou mediana temporária
            if df_temp.loc[idx, target_col] == -1:
                # Usar mediana dos valores válidos até agora
                valid_values = df_temp[df_temp[target_col] != -1][target_col]
                if len(valid_values) > 0:
                    df_temp.loc[idx, target_col] = valid_values.median()
                else:
                    df_temp.loc[idx, target_col] = df_clean[target_col].median()
            
            # Feature engineering
            df_temp_feat = engineer_features_for_imputation(df_temp, target_col)
            
            # Extrair features da última linha
            X_pred = df_temp_feat.iloc[-1:][feature_cols].fillna(0).values
            X_pred_scaled = scaler_X.transform(X_pred)
            
            # Predizer
            y_pred_scaled = stacking.predict(X_pred_scaled)
            y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()[0]
            
            # Aplicar predição
            df_imputed.loc[idx, target_col] = y_pred
            df_imputed.loc[idx, 'is_imputed'] = 1
            imputed_values.append(y_pred)
        
        # Salvar resultado
        cols_to_save = ["Data", "Atraso(ms)", "Hop_count", "Bottleneck", target_col, "is_imputed"]
        output_file = output_dir / f"{source}_stacking.csv"
        df_imputed[cols_to_save].to_csv(output_file, index=False)
        
        # Estatísticas das imputações
        imputed_array = np.array(imputed_values)
        print(f"      ✓ Stacking: {output_file}")
        print(f"      ✓ Valores imputados: {n_missing}")
        print(f"      ✓ Média: {np.mean(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Desvio: {np.std(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Min: {np.min(imputed_array)/1e6:.2f}M, Max: {np.max(imputed_array)/1e6:.2f}M")
        print(f"      ✓ Valores únicos: {len(np.unique(imputed_array))}/{n_missing}")
        
    except Exception as e:
        print(f"      ✗ Stacking falhou: {e}")
        import traceback
        traceback.print_exc()

def intelligent_imputation(file_path: Path, results: Dict, output_dir: Path):
    """
    Decide se deve imputar baseado nos resultados da avaliação.
    Se stacking for melhor, imputa com todos os métodos (baselines + stacking).
    Caso contrário, apenas salva baselines.
    """
    source = results["source"]
    analysis = analyze_stacking_performance(results["results"])
    
    print(f"\n{'='*80}")
    print(f"[ANÁLISE DE DESEMPENHO] {source}")
    print(f"{'='*80}")
    print(f"  Vitórias do Stacking: {analysis['stacking_wins']}/{analysis['total_comparisons']} ({analysis['win_rate']*100:.1f}%)")
    print(f"  RMSE Médio Stacking: {analysis['avg_stacking_rmse']:.2f}M")
    print(f"  RMSE Médio Melhor Baseline: {analysis['avg_baseline_rmse']:.2f}M")
    print(f"  Melhoria Média: {analysis['avg_improvement']:.2f}%")
    
    # ✅ Exibir tempo de predição
    if analysis.get('avg_prediction_time_per_sample') is not None:
        print(f"  ⏱️  Tempo Médio de Predição: {analysis['avg_prediction_time_per_sample']:.4f}ms/amostra")
    
    # Carregar dados originais
    df = pd.read_csv(file_path)
    target_col = "Vazao_BBR"
    
    # Verificar se há valores para imputar
    n_missing = (df[target_col] == -1).sum()
    
    if n_missing == 0:
        print(f"  ℹ️  Nenhum valor faltante para imputar")
        return
    
    print(f"  📊 {n_missing} valores faltantes encontrados")
    
    # Criar diretório de saída
    output_dir.mkdir(parents=True, exist_ok=True)
    
    if analysis['should_impute']:
        print(f"  ✅ DECISÃO: Stacking melhor - IMPUTANDO COM TODOS OS MÉTODOS")
        print(f"\n  {'─'*60}")
        
        # Imputar com baselines
        impute_with_baselines(df, target_col, output_dir, source)
        
        # Imputar com stacking
        impute_with_stacking(df, target_col, output_dir, source)
        
        print(f"  {'─'*60}")
        print(f"  ✅ Imputação concluída para {source}")
        
    else:
        print(f"  ❌ DECISÃO: Stacking NÃO foi melhor - APENAS BASELINES")
        print(f"\n  {'─'*60}")
        
        # Imputar apenas com baselines
        impute_with_baselines(df, target_col, output_dir, source)
        
        print(f"  {'─'*60}")
        print(f"  ⚠️  Stacking não aplicado para {source}")

# ========== EXECUÇÃO PRINCIPAL ==========

def main():
    """
    Executa avaliação completa e imputação inteligente em todos os arquivos CSV.
    """
    # Configuração
    data_path = Path("../../datasets/multivariada-post-process")
    results_path = Path("../../results")
    imputed_path = Path("../../datasets/multivariada-imputed-final")  # ✅ NOME ALTERADO
    
    results_path.mkdir(exist_ok=True, parents=True)
    imputed_path.mkdir(exist_ok=True, parents=True)
    
    csv_files = list(data_path.glob("*_merged.csv"))
    missing_fractions = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    
    print(f"\n{'='*80}")
    print(f"PIPELINE COMPLETO: AVALIAÇÃO + IMPUTAÇÃO INTELIGENTE")
    print(f"{'='*80}")
    print(f"Arquivos encontrados: {len(csv_files)}")
    print(f"Frações de missing: {missing_fractions}")
    print(f"Pasta de resultados: {results_path}")
    print(f"Pasta de dados imputados: {imputed_path}")
    print(f"{'='*80}\n")
    
    all_results = {}
    summary = {
        "total_files": len(csv_files),
        "processed": 0,
        "stacking_used": 0,
        "baseline_only": 0,
        "failed": 0
    }
    
    for i, file_path in enumerate(csv_files, 1):
        print(f"\n{'#'*80}")
        print(f"[{i}/{len(csv_files)}] Processando: {file_path.name}")
        print(f"{'#'*80}")
        
        try:
            # FASE 1: AVALIAÇÃO
            print(f"\n{'='*80}")
            print(f"FASE 1: AVALIAÇÃO")
            print(f"{'='*80}")
            
            result = evaluate_file(file_path, missing_fractions)
            
            if result is None:
                print(f"  ⚠️ Arquivo ignorado (dados insuficientes)")
                summary["failed"] += 1
                continue
            
            source = result["source"]
            all_results[source] = result["results"]
            
            # Salvar resultados parciais da avaliação
            evaluation_file = results_path / "metrics_summary_final.json"  # ✅ NOME ALTERADO
            with open(evaluation_file, 'w') as f:
                json.dump(all_results, f, indent=4)
            
            print(f"\n  💾 Avaliação salva em: {evaluation_file}")
            
            # FASE 2: IMPUTAÇÃO INTELIGENTE
            print(f"\n{'='*80}")
            print(f"FASE 2: IMPUTAÇÃO INTELIGENTE")
            print(f"{'='*80}")
            
            intelligent_imputation(file_path, result, imputed_path)
            
            # Atualizar sumário
            analysis = analyze_stacking_performance(result["results"])
            summary["processed"] += 1
            if analysis['should_impute']:
                summary["stacking_used"] += 1
            else:
                summary["baseline_only"] += 1
            
            print(f"\n  ✅ Concluído: {source}")
            
        except Exception as e:
            print(f"\n  ❌ Erro processando {file_path.name}: {e}")
            import traceback
            traceback.print_exc()
            summary["failed"] += 1
            continue
    
    # RELATÓRIO FINAL
    print(f"\n{'='*80}")
    print(f"RELATÓRIO FINAL")
    print(f"{'='*80}")
    print(f"Total de arquivos: {summary['total_files']}")
    print(f"Processados com sucesso: {summary['processed']}")
    print(f"  ├─ Stacking usado: {summary['stacking_used']} ({summary['stacking_used']/max(1,summary['processed'])*100:.1f}%)")
    print(f"  └─ Apenas baselines: {summary['baseline_only']} ({summary['baseline_only']/max(1,summary['processed'])*100:.1f}%)")
    print(f"Falhas: {summary['failed']}")
    print(f"\n📁 Resultados salvos em:")
    print(f"  ├─ Avaliação: {results_path / 'metrics_summary_final.json'}")  # ✅ NOME ALTERADO
    print(f"  └─ Dados imputados: {imputed_path}")
    print(f"{'='*80}\n")
    
    # Salvar sumário
    summary_file = results_path / "imputation_summary_final.json"  # ✅ NOME ALTERADO
    with open(summary_file, 'w') as f:
        json.dump(summary, f, indent=4)
    
    print(f"📊 Sumário salvo em: {summary_file}\n")

if __name__ == "__main__":
    main()


PIPELINE COMPLETO: AVALIAÇÃO + IMPUTAÇÃO INTELIGENTE
Arquivos encontrados: 615
Frações de missing: [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
Pasta de resultados: ..\..\results
Pasta de dados imputados: ..\..\datasets\multivariada-imputed-final


################################################################################
[1/615] Processando: ac-am_merged.csv
################################################################################

FASE 1: AVALIAÇÃO

[AVALIANDO] ac-am_merged.csv
  [DADOS LIMPOS] 1326 amostras válidas (removidos 885 com -1)

  ────────────────────────────────────────────────────────────
  FRAÇÃO DE MISSING: 20%
  ────────────────────────────────────────────────────────────
  [BASELINE] Avaliando fração 20%
    Máscara aplicada: 265/1326 amostras (20%)
  [STACKING] Avaliando com missing_fraction=20%
    Máscara aplicada: 265/1326 amostras (20%)
    Treino: 1061 amostras, Teste: 265 amostras
    Modelo: 1061 -> 265 previsões
    ✅ RMSE: 44.54M, R²: 0.8669

  ────